In [1]:
import trimesh
import pyrender
import os.path as ospath
import trimesh.sample as sample
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
person_trimesh = trimesh.load(ospath.join(ospath.curdir, "frame101.obj"))
mesh = pyrender.Mesh.from_trimesh(person_trimesh)
scene = pyrender.Scene()
scene.add(mesh)

In [3]:
# pyrender.Viewer(scene, use_raymond_lighting=True)

In [4]:
points_even, idx_even = sample.sample_surface_even(person_trimesh, 500)

In [5]:
scene = pyrender.Scene()
scene.add(mesh)

In [6]:
center = mesh.centroid
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
camera_pose = np.array([
   [1.0,  0.0, 0.0, 0 + center[0]],
   [0.0,  1.0, 0.0, 0 + center[1]],
   [0.0,  0.0, 1.0, 2 + center[2]],
   [0.0,  0.0, 0.0, 1.0],
])

In [7]:
scene.add(camera, pose=camera_pose)
light = pyrender.SpotLight(color=np.ones(3), intensity=1.0,
                           innerConeAngle=np.pi/16.0,
                           outerConeAngle=np.pi/6.0)
scene.add(light, pose=camera_pose)
r = pyrender.OffscreenRenderer(1024, 1024)
color, depth = r.render(scene)


In [8]:
cv2.imshow("first", color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def rotation_matrix_x(angle_rad):
    return np.array([
        [1, 0, 0],
        [0, np.cos(angle_rad), -np.sin(angle_rad)],
        [0, np.sin(angle_rad), np.cos(angle_rad)]
    ])

def rotation_matrix_y(angle_rad):
    return np.array([
        [np.cos(angle_rad), 0, np.sin(angle_rad)],
        [0, 1, 0],
        [-np.sin(angle_rad), 0, np.cos(angle_rad)]
    ])

def rotation_matrix_z(angle_rad):
    return np.array([
        [np.cos(angle_rad), -np.sin(angle_rad), 0],
        [np.sin(angle_rad), np.cos(angle_rad), 0],
        [0, 0, 1]
    ])

In [23]:
angle_x_rad = 45 * np.pi / 180
angle_y_rad = 45 * np.pi / 180
angle_z_rad = 45 * np.pi / 180

R_x = rotation_matrix_x(angle_x_rad)
R_y = rotation_matrix_y(angle_y_rad)
R_z = rotation_matrix_z(angle_z_rad)


R = np.dot(R_z, R_y, R_x)

In [24]:
# Rotating the camera's orientation
new_camera_pose = np.copy(camera_pose)
new_camera_pose[:3, :3] = np.dot(R, camera_pose[:3, :3])

# Rotating the camera's position
old_position = np.array([0, 0, 2])
new_position = np.dot(R, old_position)
new_camera_pose[:3, 3] = new_position + center

In [25]:
scene.clear()
scene.add(mesh)
scene.add(camera, pose=new_camera_pose)
scene.add(light, pose=new_camera_pose)
color, depth = r.render(scene)
# cv2.imshow("b", color)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("backside.png", color)

True

In [17]:
point_cloud = trimesh.points.PointCloud(points_even)

In [60]:
point_mesh = pyrender.Mesh.from_points(points_even)
scene.clear()
scene.add(point_mesh)
scene.add(camera, pose=camera_pose)
scene.add(light, pose=camera_pose)
color, depth = r.render(scene)
cv2.imshow("c", color)
cv2.waitKey(0)
cv2.destroyAllWindows()